In [169]:
import matplotlib.pyplot as plt 
import pandas as pd 

from sklearn.datasets import fetch_california_housing 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error , r2_score 


In [170]:
data = fetch_california_housing()

In [171]:
data

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [172]:
X= pd.DataFrame(data.data , columns = data.feature_names)
y= pd.DataFrame( data.target , columns = ['MedHouseVal'])

In [173]:
X.isnull().sum().sum()

0

In [174]:
X_train , X_test , y_train , y_test = train_test_split( X , y , test_size=0.2 , random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [175]:
import torch
from torch.utils.data import DataLoader , TensorDataset 
import torch.nn as nn 
import torch.optim as optim 


### 1. converting data to torch version


In [176]:
X_train_tensor = torch.tensor(X_train , dtype=torch.float32)
y_train_tensor= torch.tensor(y_train.values , dtype = torch.float32)
X_test_tensor = torch.tensor(X_test , dtype=torch.float32)
y_test_tensor= torch.tensor(y_test.values , dtype = torch.float32)


### 2. converting dataset and load it to Tensor form

In [177]:
train_loader = DataLoader(TensorDataset(X_train_tensor , y_train_tensor) , batch_size = 32 , shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor , y_test_tensor) , batch_size = 32, shuffle=False)

In [178]:
class RegressionNN(nn.Module): 
    def __init__ (self , input_size=8): 
        super(RegressionNN , self).__init__()
        self.fc1 = nn.Linear(input_size , 128)
        self.fc2 = nn.Linear(128 , 64)
        self.fc3 = nn.Linear(64 , 32) 
        self.fc4 = nn.Linear(32 , 1)
    def forward (self , x): 
        x=torch.relu(self.fc1(x))
        x=torch.relu(self.fc2(x))
        x=torch.relu(self.fc3(x))
        x= self.fc4(x)
        return x
        

In [179]:
pytorch_model = RegressionNN()

criterion = nn.MSELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.001)

In [180]:
from torchsummary import summary
print('pytorch model summary:')
summary( pytorch_model , input_size=(8,))

pytorch model summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           1,152
            Linear-2                   [-1, 64]           8,256
            Linear-3                   [-1, 32]           2,080
            Linear-4                    [-1, 1]              33
Total params: 11,521
Trainable params: 11,521
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


In [181]:
def train_regression(train_loader , model , criterion , optimizer ,num_epochs= 10): 
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0
        for inputs , targets in train_loader: 
                optimizer.zero_grad()
                output=model(inputs) 
                loss = criterion(output , targets)
                loss.backward() # backpropagation
                optimizer.step()
                running_loss+=loss.item()
        print(f"epoch: {epoch+1}/{num_epochs} , loss: {running_loss/len(train_loader)}")

train_regression(train_loader , pytorch_model , criterion , optimizer , num_epochs=100)


epoch: 1/100 , loss: 0.9002845763236054
epoch: 2/100 , loss: 0.3987004355953414
epoch: 3/100 , loss: 0.3715012566179268
epoch: 4/100 , loss: 0.3504860571535059
epoch: 5/100 , loss: 0.33810010379136995
epoch: 6/100 , loss: 0.3279580239697482
epoch: 7/100 , loss: 0.3160499163087486
epoch: 8/100 , loss: 0.3101043571364279
epoch: 9/100 , loss: 0.30635834290364455
epoch: 10/100 , loss: 0.2997486053049102
epoch: 11/100 , loss: 0.29255258667376616
epoch: 12/100 , loss: 0.28665958523981333
epoch: 13/100 , loss: 0.2897123590602662
epoch: 14/100 , loss: 0.28180852717207383
epoch: 15/100 , loss: 0.2824140025966967
epoch: 16/100 , loss: 0.2817397397733474
epoch: 17/100 , loss: 0.282348923081922
epoch: 18/100 , loss: 0.2790659776376199
epoch: 19/100 , loss: 0.2737309899210006
epoch: 20/100 , loss: 0.2707777785844812
epoch: 21/100 , loss: 0.26907540163270727
epoch: 22/100 , loss: 0.26746998546832756
epoch: 23/100 , loss: 0.27704542206759136
epoch: 24/100 , loss: 0.2646346707330194
epoch: 25/100 , lo

In [182]:
def eval_regression(test_loader , model):
        model.eval()
        test_loss = 0.0 
        with torch.no_grad():
            for X_batch , y_batch in test_loader: 
               output= model(X_batch) 
               loss = criterion(output , y_batch)
               test_loss += loss.item()
        return test_loss/len(test_loader)
    
test_loss = eval_regression(test_loader , pytorch_model)
print(f"Test loss: {test_loss}")

Test loss: 0.2787007662330487
